In [2]:
import numpy as np
import math
import molecule_predictionv2_0 as mp

In [3]:
import tensorflow as tf

In [4]:
dat = mp.molecule_prediction_data_wrapper()

Reading molecule structures


Done Reading molecule structures
Reading molecule prop


Done Reading molecule prop
Reading Coulomb_mat


done reading coulomb mat
Reading distance_mat


done reading distance mat


In [7]:
atom_vector_size = 1024
num_atoms = 30
num_timesteps = 5
hidden_unit_size = 256
batchSize = 32
num_epochs = 200
batch_gen = dat.batch_gen
dataset = tf.data.Dataset.\
    from_generator(batch_gen.generate, (tf.float32,tf.float32, tf.float32),
                   output_shapes= (tf.TensorShape([30,1024]),
                                   tf.TensorShape([30,30]),
                                   tf.TensorShape([3])))                                                     
dataset = dataset.shuffle(buffer_size = batchSize*10) 
dataset = dataset.repeat(num_epochs).batch(batchSize)
dataset = dataset.prefetch(buffer_size = 2)
data_source =  dataset.make_one_shot_iterator()
atoms_vector_batch, ad_matrix , batch_y = data_source.get_next()



In [8]:
with tf.variable_scope('message_transform_network'):
    hidden1 = tf.keras.layers.Dense(2048,activation='relu')
    hidden1.build((None,atom_vector_size))
    hidden2 = tf.keras.layers.Dense(2048,activation='tanh')
    hidden2.build((None,2048))
    hidden3 = tf.keras.layers.Dense(1024,activation='relu')
    hidden3.build((None,2048))
    out_message = tf.keras.layers.Dense(atom_vector_size)
    out_message.build((None, 1024))
def apply_edge_neural_network_transform_to_batch(batch_input):
    input = tf.reshape(batch_input, [-1, atom_vector_size])
    out = apply_edge_neural_network_transform(input)
    return tf.reshape(out, [batchSize, num_atoms, atom_vector_size])    
def apply_edge_neural_network_transform(input):
    return out_message.apply(hidden3.apply(hidden2.apply(hidden1.apply(input))))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
def get_messages_batch(ordered_atoms_vector_batch, adjancency_matrix_batch):
    tranformed_batch = apply_edge_neural_network_transform_to_batch(
        ordered_atoms_vector_batch)
    return tf.matmul(adjancency_matrix_batch,tranformed_batch)
def get_messages(ordered_atoms_vector, adjacency_matrix):
    transformed = apply_edge_neural_network_transform(ordered_atoms_vector)
    return tf.matmul(adjacency_matrix,transformed)

In [10]:
with tf.variable_scope("RecusiveUnitLTSM"):
    shared_lstm_cell =  tf.keras.layers.LSTMCell(hidden_unit_size);
def iterate_time_step(states_vectors_t, messages):
    (outputs_t, states_vectors_t_1) = \
        shared_lstm_cell(messages, states_vectors_t)
    return outputs_t, states_vectors_t_1

In [11]:
initial_state1 = tf.Variable(np.random.normal(size=(1,hidden_unit_size)),
                            trainable=True,dtype=tf.float32);
initial_state2 = tf.Variable(np.random.normal(size=(1,hidden_unit_size)),
                            trainable=True,dtype=tf.float32);
initial_states1 = tf.reshape(tf.tile(initial_state1, (1,num_atoms*batchSize)),
                            [num_atoms*batchSize, hidden_unit_size])
initial_states2 = tf.reshape(tf.tile(initial_state2, (1,num_atoms*batchSize)),
                            [num_atoms*batchSize, hidden_unit_size])
initial_states = (initial_states1,initial_states2)

In [12]:
def extract_atom_vectors_ad_matrix(input_mat):
    num_atoms = tf.shape(input_mat)[0]
    return tf.slice(input_mat,[0,0],[num_atoms,atom_vector_size]),\
           tf.slice(input_mat,[0,atom_vector_size],[num_atoms,num_atoms])

In [13]:
def graph_neural_network(concatenated_input_mat):
    global initial_states
    initial_output,ad_matrix = \
        extract_atom_vectors_ad_matrix(concatenated_input_mat)
    outputs_x = initial_output
    states_vectors_x = initial_states
    for i in range(num_timesteps):
        messages = get_messages(outputs_x, ad_matrix)
        outputs_x, states_vectors_x = iterate_time_step(states_vectors_x,messages)    
    final_outputs = outputs_x
    out = tf.reduce_sum(final_outputs, axis=0)
    return out 

In [14]:
def graph_neural_network_batch(atoms_vector_batch, ad_matrix):
    global initial_states   
    outputs_x = atoms_vector_batch
    states_vectors_x = initial_states
    for i in range(num_timesteps):
        messages = get_messages_batch(outputs_x, ad_matrix)
        messages = tf.reshape(messages, [-1, atom_vector_size])
        outputs_x, states_vectors_x = iterate_time_step(states_vectors_x,messages)    
        outputs_x = tf.reshape(outputs_x,[batchSize,num_atoms,atom_vector_size])
    final_outputs = tf.reshape(outputs_x,[batchSize,num_atoms,atom_vector_size])
    out = tf.reduce_sum(final_outputs,axis=1)
    return out

In [15]:
final_outputs=graph_neural_network_batch(atoms_vector_batch, ad_matrix)
prediction = tf.keras.layers.Dense(3)(final_outputs)
with tf.name_scope("loss"):
    loss = tf.losses.mean_squared_error(batch_y, prediction)


Instructions for updating:
Use tf.cast instead.


In [16]:
learning_rate = 0.001
with tf.name_scope("train"):
    global training_op
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

Instructions for updating:
Use tf.cast instead.


In [17]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    init.run()
    #for epoch in range(num_epochs):
        #for iteration in range(batch_gen.samples// batchSize):
    _,loss_value = sess.run([training_op,loss])          
    #print("Epoch " + str(epoch) + " Step " + str(iteration) + " loss " + str(loss_value))
        #if epoch % 20 == 0:
            #save_path = saver.save(sess, "../models/graph_model" + str(epoch) + ".ckpt")            

E:\Kaggle\Molecule\MoleculePrediction\molecule_predictionv2_0.py:171: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[insertHere] = array


In [18]:
loss_value

18.21106

In [22]:
[print(n.name) for n in tf.get_default_graph().as_graph_def().node]

tensors/component_0
buffer_size
seed
seed2
count
batch_size
drop_remainder
buffer_size_1
OneShotIterator
IteratorToStringHandle
IteratorGetNext
tensors_1/component_0
buffer_size_2
seed_1
seed2_1
count_1
batch_size_1
drop_remainder_1
buffer_size_3
OneShotIterator_1
IteratorToStringHandle_1
IteratorGetNext_1
message_transform_network/kernel/Initializer/random_uniform/shape
message_transform_network/kernel/Initializer/random_uniform/min
message_transform_network/kernel/Initializer/random_uniform/max
message_transform_network/kernel/Initializer/random_uniform/RandomUniform
message_transform_network/kernel/Initializer/random_uniform/sub
message_transform_network/kernel/Initializer/random_uniform/mul
message_transform_network/kernel/Initializer/random_uniform
message_transform_network/kernel
message_transform_network/kernel/IsInitialized/VarIsInitializedOp
message_transform_network/kernel/Assign
message_transform_network/kernel/Read/ReadVariableOp
message_transform_network/bias/Initializer/z


train/gradients/lstm_cell_3/add_7_grad/tuple/control_dependency_1
train/gradients/lstm_cell_3/mul_2_grad/Mul
train/gradients/lstm_cell_3/mul_2_grad/Mul_1
train/gradients/lstm_cell_3/mul_2_grad/tuple/group_deps
train/gradients/lstm_cell_3/mul_2_grad/tuple/control_dependency
train/gradients/lstm_cell_3/mul_2_grad/tuple/control_dependency_1
train/gradients/lstm_cell_3/mul_3_grad/Mul
train/gradients/lstm_cell_3/mul_3_grad/Mul_1
train/gradients/lstm_cell_3/mul_3_grad/tuple/group_deps
train/gradients/lstm_cell_3/mul_3_grad/tuple/control_dependency
train/gradients/lstm_cell_3/mul_3_grad/tuple/control_dependency_1
train/gradients/lstm_cell_3/mul_4_grad/Shape
train/gradients/lstm_cell_3/mul_4_grad/Shape_1
train/gradients/lstm_cell_3/mul_4_grad/BroadcastGradientArgs
train/gradients/lstm_cell_3/mul_4_grad/Mul
train/gradients/lstm_cell_3/mul_4_grad/Sum
train/gradients/lstm_cell_3/mul_4_grad/Reshape
train/gradients/lstm_cell_3/mul_4_grad/Mul_1
train/gradients/lstm_cell_3/mul_4_grad/Sum_1
train/gra

train/gradients/lstm_cell/mul_4_grad/Shape_1
train/gradients/lstm_cell/mul_4_grad/BroadcastGradientArgs
train/gradients/lstm_cell/mul_4_grad/Mul
train/gradients/lstm_cell/mul_4_grad/Sum
train/gradients/lstm_cell/mul_4_grad/Reshape
train/gradients/lstm_cell/mul_4_grad/Mul_1
train/gradients/lstm_cell/mul_4_grad/Sum_1
train/gradients/lstm_cell/mul_4_grad/Reshape_1
train/gradients/lstm_cell/mul_4_grad/tuple/group_deps
train/gradients/lstm_cell/mul_4_grad/tuple/control_dependency
train/gradients/lstm_cell/mul_4_grad/tuple/control_dependency_1
train/gradients/lstm_cell/clip_by_value_1_grad/Shape
train/gradients/lstm_cell/clip_by_value_1_grad/Shape_1
train/gradients/lstm_cell/clip_by_value_1_grad/Shape_2
train/gradients/lstm_cell/clip_by_value_1_grad/zeros/Const
train/gradients/lstm_cell/clip_by_value_1_grad/zeros
train/gradients/lstm_cell/clip_by_value_1_grad/GreaterEqual
train/gradients/lstm_cell/clip_by_value_1_grad/BroadcastGradientArgs
train/gradients/lstm_cell/clip_by_value_1_grad/Selec

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [23]:
tf.trainable_variables()

[<tf.Variable 'message_transform_network/kernel:0' shape=(1024, 2048) dtype=float32>,
 <tf.Variable 'message_transform_network/bias:0' shape=(2048,) dtype=float32>,
 <tf.Variable 'message_transform_network/kernel_1:0' shape=(2048, 2048) dtype=float32>,
 <tf.Variable 'message_transform_network/bias_1:0' shape=(2048,) dtype=float32>,
 <tf.Variable 'message_transform_network/kernel_2:0' shape=(2048, 1024) dtype=float32>,
 <tf.Variable 'message_transform_network/bias_2:0' shape=(1024,) dtype=float32>,
 <tf.Variable 'message_transform_network/kernel_3:0' shape=(1024, 1024) dtype=float32>,
 <tf.Variable 'message_transform_network/bias_3:0' shape=(1024,) dtype=float32>,
 <tf.Variable 'Variable:0' shape=(1, 1024) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(1, 1024) dtype=float32_ref>,
 <tf.Variable 'lstm_cell/kernel:0' shape=(1024, 4096) dtype=float32>,
 <tf.Variable 'lstm_cell/recurrent_kernel:0' shape=(1024, 4096) dtype=float32>,
 <tf.Variable 'lstm_cell/bias:0' shape=(4096,) dty